# Read Cleaned Delta Tables from S3

This notebook reads cleaned reddit data from the `{subreddit}_clean` Delta tables stored on S3 using Polars and deltalake.

In [1]:
import os
import json
import yaml
import polars as pl
from deltalake import DeltaTable
from datetime import datetime

# Function to find files in the workspace
def find_file(filename, search_paths):
    """Search for a file in specified paths and parent directories"""
    # Check specific paths first
    for path in search_paths:
        if os.path.exists(path):
            return path
    
    # Walk up from CWD
    curr = os.getcwd()
    while True:
        f = os.path.join(curr, filename)
        if os.path.exists(f):
            return f
        parent = os.path.dirname(curr)
        if parent == curr:
            break
        curr = parent
    return None

def read_config_creds():
    """Read credentials and configuration files"""
    print(f"Current working directory: {os.getcwd()}")
    
    # Creds paths to search
    creds_paths = [
        "creds.json",
        "redditStreaming/creds.json",
        "/home/steven/reddit-streaming/creds.json",
        "/opt/workspace/creds.json",
    ]
    
    creds_file = find_file("creds.json", creds_paths)
    if not creds_file:
         raise FileNotFoundError("Could not find creds.json")
    
    print(f"Found credentials at: {creds_file}")
    with open(creds_file, "r") as f:
        creds = json.load(f)

    # Config paths to search
    config_paths = [
        "config.yaml",
        "redditStreaming/config.yaml",
        "/home/steven/reddit-streaming/config.yaml",
        "/opt/workspace/config.yaml"
    ]
    
    config_file = find_file("config.yaml", config_paths)
    if not config_file:
         raise FileNotFoundError("Could not find config.yaml")
         
    print(f"Found config at: {config_file}")
    with open(config_file, "r") as f:
        config = yaml.safe_load(f)
        
    return creds, config

# Load configuration and credentials
creds, config = read_config_creds()

# Extract values
aws_client = creds.get("aws_client")
aws_secret = creds.get("aws_secret")
subreddits = config.get("subreddit", [])
bucket_name = "reddit-streaming-stevenhurwitt-2" 

print(f"Subreddits to process: {subreddits}")
print(f"S3 Bucket: {bucket_name}")

Current working directory: /home/steven/reddit-streaming/redditStreaming/src/notebooks
Found credentials at: /home/steven/reddit-streaming/creds.json
Found config at: config.yaml
Subreddits to process: ['technology', 'ProgrammerHumor', 'news', 'worldnews']
S3 Bucket: reddit-streaming-stevenhurwitt-2


## Configure AWS for Delta Lake

In [2]:
# Configure AWS credentials for Delta Lake S3 access
storage_options = {
    "AWS_ACCESS_KEY_ID": aws_client,
    "AWS_SECRET_ACCESS_KEY": aws_secret,
    "AWS_REGION": "us-east-2"  # Adjust if your bucket is in a different region
}

print("AWS credentials configured for Delta Lake.")

AWS credentials configured for Delta Lake.


## Read Cleaned Delta Tables from S3

Read each `{subreddit}_clean` delta table from S3

In [3]:
# Read cleaned Delta tables for each subreddit
dfs = {}

for sub in subreddits:
    path = f"s3://{bucket_name}/{sub}_clean"
    print(f"Reading from: {path}")
    try:
        # Read Delta table using deltalake and convert to Polars
        dt = DeltaTable(path, storage_options=storage_options)
        df = dt.to_pyarrow_table()
        df = pl.from_arrow(df)
        dfs[sub] = df
        count = len(df)
        print(f"✓ Successfully read {sub}_clean: {count:,} records")
    except Exception as e:
        print(f"✗ Error reading {sub}_clean: {str(e)}")

print(f"\nTotal tables loaded: {len(dfs)}")

Reading from: s3://reddit-streaming-stevenhurwitt-2/technology_clean
✓ Successfully read technology_clean: 3,354 records
Reading from: s3://reddit-streaming-stevenhurwitt-2/ProgrammerHumor_clean
✓ Successfully read ProgrammerHumor_clean: 2,350 records
Reading from: s3://reddit-streaming-stevenhurwitt-2/news_clean
✓ Successfully read news_clean: 2,186 records
Reading from: s3://reddit-streaming-stevenhurwitt-2/worldnews_clean
✓ Successfully read worldnews_clean: 4,990 records

Total tables loaded: 4


## Display Individual Subreddit Data

View data from each subreddit separately

In [4]:
# Display sample data from each subreddit
for sub, df in dfs.items():
    print(f"\n{'='*60}")
    print(f"Subreddit: r/{sub}")
    print(f"{'='*60}")
    
    # Add human-readable timestamp
    display_df = df.with_columns(
        pl.from_epoch("created_utc", time_unit="s").alias("created_time")
    )
    
    # Show schema
    print("\nSchema:")
    print(display_df.schema)
    
    # Show sample records (most recent)

    print(f"\nSample records (most recent):")    
    print(display_df.sort("created_utc", descending=True).head(10))


Subreddit: r/technology

Schema:
Schema({'approved_at_utc': Datetime(time_unit='us', time_zone=None), 'subreddit': String, 'selftext': String, 'author_fullname': String, 'saved': Boolean, 'mod_reason_title': String, 'gilded': Int32, 'clicked': Boolean, 'title': String, 'subreddit_name_prefixed': String, 'hidden': Boolean, 'pwls': Int32, 'link_flair_css_class': String, 'downs': Int32, 'thumbnail_height': Int32, 'top_awarded_type': String, 'hide_score': Boolean, 'name': String, 'quarantine': Boolean, 'link_flair_text_color': String, 'upvote_ratio': Float32, 'author_flair_background_color': String, 'ups': Int32, 'total_awards_received': Int32, 'thumbnail_width': Int32, 'author_flair_template_id': String, 'is_original_content': Boolean, 'secure_media': String, 'is_reddit_media_domain': Boolean, 'is_meta': Boolean, 'category': String, 'link_flair_text': String, 'can_mod_post': Boolean, 'score': Int32, 'approved_by': String, 'is_created_from_ads_ui': Boolean, 'author_premium': Boolean, 'thu

## Union All Subreddits and Sort by Recency

Combine all subreddit data into a single DataFrame and sort by most recent posts

In [5]:
# Union all dataframes and sort by recency
if dfs:
    # Concatenate all dataframes
    all_dfs = list(dfs.values())
    combined_df = pl.concat(all_dfs)
    
    # Sort by created_utc (most recent first)
    sorted_df = combined_df.sort("created_utc", descending=True)
    
    # Add human-readable timestamp
    display_df = sorted_df.with_columns(
        pl.from_epoch("created_utc", time_unit="s").alias("created_time")
    )
    
    total_count = len(sorted_df)
    print(f"Total records across all subreddits: {total_count:,}")
    print(f"\nMost recent posts across all subreddits:")
    
    # Display relevant columns
    print(display_df.select([
            "subreddit", 
            "created_time", 
            "title", 
            "score", 
            "author",
            "num_comments"

        ]).head(50)    
    )
    
else:
    print("No data available from any subreddit.")

Total records across all subreddits: 12,880

Most recent posts across all subreddits:
shape: (50, 6)
┌────────────┬─────────────────┬──────────────────────┬───────┬─────────────────────┬──────────────┐
│ subreddit  ┆ created_time    ┆ title                ┆ score ┆ author              ┆ num_comments │
│ ---        ┆ ---             ┆ ---                  ┆ ---   ┆ ---                 ┆ ---          │
│ str        ┆ datetime[μs]    ┆ str                  ┆ i32   ┆ str                 ┆ i32          │
╞════════════╪═════════════════╪══════════════════════╪═══════╪═════════════════════╪══════════════╡
│ news       ┆ +48007-03-05    ┆ How the world has    ┆ 4     ┆ GroundbreakingArm17 ┆ 0            │
│            ┆ 21:58:03.044864 ┆ reacted to U…        ┆       ┆ 3                   ┆              │
│ worldnews  ┆ +48003-02-13    ┆ How the world has    ┆ 1     ┆ GroundbreakingArm17 ┆ 1            │
│            ┆ 10:24:43.044864 ┆ reacted to U…        ┆       ┆ 3                   ┆      

## Summary Statistics

View summary statistics for each subreddit

In [6]:
# Summary statistics by subreddit
if dfs:
    print("Summary by subreddit:")
    print(f"{'Subreddit':<20} {'Records':>15} {'Avg Score':>12} {'Avg Comments':>15}")
    print("="*65)
    
    for sub, df in dfs.items():
        count = len(df)
        avg_score = df.select(pl.col("score").mean()).item()
        avg_comments = df.select(pl.col("num_comments").mean()).item()
        
        print(f"{sub:<20} {count:>15,} {avg_score:>12.2f} {avg_comments:>15.2f}")

Summary by subreddit:
Subreddit                    Records    Avg Score    Avg Comments
technology                     3,354         2.42            0.37
ProgrammerHumor                2,350         4.36            0.37
news                           2,186        55.11            5.11
worldnews                      4,990         5.70            1.15


In [ ]:
# Clean up (nothing needed for Polars)

## Post count by date/subreddit

In [7]:
display_df.group_by("subreddit", "date").agg([
    pl.len().alias("post_count")]).sort("date", descending=True).head(20)

subreddit,date,post_count
str,date,u32
"""news""",2026-03-01,15
"""technology""",2026-03-01,3
"""ProgrammerHumor""",2026-03-01,3
"""worldnews""",2026-03-01,20
"""technology""",2026-02-28,59
…,…,…
"""ProgrammerHumor""",2026-02-26,31
"""news""",2026-02-25,32
"""technology""",2026-02-25,63


In [8]:
display_df.head(10)

approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,subreddit_name_prefixed,hidden,pwls,link_flair_css_class,downs,thumbnail_height,top_awarded_type,hide_score,name,quarantine,link_flair_text_color,upvote_ratio,author_flair_background_color,ups,total_awards_received,thumbnail_width,author_flair_template_id,is_original_content,secure_media,is_reddit_media_domain,is_meta,category,link_flair_text,can_mod_post,score,approved_by,is_created_from_ads_ui,author_premium,…,locked,author_flair_text,visited,removed_by,mod_note,distinguished,subreddit_id,author_is_blocked,mod_reason_by,num_reports,removal_reason,link_flair_background_color,id,is_robot_indexable,report_reasons,author,discussion_type,num_comments,send_replies,whitelist_status,contest_mode,author_patreon_flair,author_flair_text_color,permalink,parent_whitelist_status,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video,date,year,month,day,created_time
datetime[μs],str,str,str,bool,str,i32,bool,str,str,bool,i32,str,i32,i32,str,bool,str,bool,str,f32,str,i32,i32,i32,str,bool,str,bool,bool,str,str,bool,i32,str,bool,bool,…,bool,str,bool,str,str,str,str,bool,str,i32,str,str,str,bool,str,str,str,i32,bool,str,bool,bool,str,str,str,bool,str,i32,datetime[μs],i32,str,bool,date,i32,i8,i8,datetime[μs]
null,"""news""","""""","""t2_ei935xnv""",false,null,0,false,"""How the world has reacted to U…","""r/news""",false,6,null,0,null,null,true,"""t3_1rho3m9""",false,"""dark""",1.0,null,4,0,null,null,false,null,false,false,null,null,false,4,null,false,false,…,false,null,false,null,null,null,"""t5_2qh3l""",false,null,null,null,"""""","""1rho3m9""",true,null,"""GroundbreakingArm173""",null,0,true,null,false,false,null,"""/r/news/comments/1rho3m9/how_t…",null,false,"""https://www.bbc.co.uk/news/art…",31212199,2026-03-01 04:43:44,0,null,false,2026-03-01,2026,3,1,+48007-03-05 21:58:03.044864
null,"""worldnews""","""""","""t2_ei935xnv""",false,null,0,false,"""How the world has reacted to U…","""r/worldnews""",false,7,"""normal""",0,78,null,true,"""t3_1rho1xf""",false,null,0.67,null,1,0,140,null,false,null,false,false,null,"""Dynamic Paywall""",false,1,null,false,false,…,false,null,false,null,null,null,"""t5_2qh13""",false,null,null,null,null,"""1rho1xf""",true,null,"""GroundbreakingArm173""",null,1,true,null,false,false,null,"""/r/worldnews/comments/1rho1xf/…",null,false,"""https://www.bbc.co.uk/news/art…",47109817,2026-03-01 04:41:36,0,null,false,2026-03-01,2026,3,1,+48003-02-13 10:24:43.044864
null,"""worldnews""","""""","""t2_vgozecboq""",false,null,0,false,"""South Korea president calls on…","""r/worldnews""",false,7,"""northkorea""",0,73,null,true,"""t3_1rhnopa""",false,null,1.0,null,1,0,140,null,false,null,false,false,null,"""North Korea""",false,1,null,false,false,…,false,null,false,null,null,null,"""t5_2qh13""",false,null,null,null,null,"""1rhnopa""",true,null,"""Little-Chemical5006""",null,1,true,null,false,false,null,"""/r/worldnews/comments/1rhnopa/…",null,false,"""https://www.reuters.com/world/…",47109724,2026-03-01 04:22:24,0,null,false,2026-03-01,2026,3,1,+47966-08-13 02:24:43.044864
null,"""worldnews""","""""","""t2_lakfqom6i""",false,null,0,false,"""Blasts continue in Kabul amid …","""r/worldnews""",false,7,null,0,78,null,true,"""t3_1rhnmz6""",false,"""dark""",1.0,null,1,0,140,null,false,null,false,false,null,null,false,1,null,false,false,…,false,null,false,null,null,null,"""t5_2qh13""",false,null,null,null,"""""","""1rhnmz6""",true,null,"""Cybertronian1512""",null,1,true,null,false,false,null,"""/r/worldnews/comments/1rhnmz6/…",null,false,"""https://www.thehindu.com/news/…",47109718,2026-03-01 04:20:16,0,null,false,2026-03-01,2026,3,1,+47962-07-23 14:51:23.044864
null,"""worldnews""","""""","""t2_qhdi4nf35""",false,null,0,false,"""State of Palestine strongly co…","""r/worldnews""",false,7,"""palestisrael""",0,null,null,true,"""t3_1rhnlou""",false,null,1.0,null,1,0,null,null,false,null,false,false,null,"""Israel/Palesti